# Loading modules

In [1]:
import xarray as xr
import numpy as np
import sys
from glob import glob
import gc

sys.path.append('../../')
"""
local scripts, if loading from a different directory include that with a '.' between
directory name and script name
"""
from tropical_PODs.PODs.POD_utils import calculate_saturation_specific_humidity

# Define input directories and file names

In [3]:
# Years to analyze
start_year = (2010)
end_year = (2010)

################
###.  ERA5.  ###
################

# Atmosphere

ifile_specific_humidity = '../../tropical_PODs/data/shum.2p5.*.nc' # ERA5 Specific Humidity
ifile_temperature = '../../tropical_PODs/data/air.2p5.*.nc' # ERA5 Temperature
ifile_surface_pressure = '../../tropical_PODs/data/pres.sfc.2p5.*.nc' # ERA5 Surface Pressure
ifile_precipitation = '../../tropical_PODs/data/3B-DAY.MS.MRG.3IMERG.V06.*' # IMERG Precipitation

# Land
ifile_land_frac = '../../tropical_PODs/data/land_sea_mask.erai.2p5.nc' # ERAi Land Fraction 


# Define output directories and file names

In [4]:
# Output directory for datasets string list
odir_datasets_string_list = ['../../tropical_PODs/examples/ofiles_examples/'] # ERA5 2p5_1d

# Driver for calculations

In [10]:
# Define constants
    
g = 9.8 # [m s^-2]

#########################################
# Define paths of files we wish to load #
#########################################
    
# glob expands paths with * to a list of files, like the unix shell #

paths_specific_humidity = glob(ifile_specific_humidity)
paths_temperature = glob(ifile_temperature)
paths_surface_pressure = glob(ifile_surface_pressure)
paths_precipitation = glob(ifile_precipitation)
paths_land = glob(ifile_land_frac)
        


In [11]:
for year in range(start_year, end_year + 1):
        
    print(year)
    
    # Define year strings #
        
    current_year_string = str(year)
            
    # Limit paths #
        
    year_limited_paths_specific_humidity = []
    year_limited_paths_temperature = []
    year_limited_paths_surface_pressure = []
    year_limited_paths_precipitation = []
            
    for string in paths_specific_humidity:
                        
        if (current_year_string in string):
                
            year_limited_paths_specific_humidity += [string]
            
    for string in paths_temperature:
                        
        if (current_year_string in string):
                
            year_limited_paths_temperature += [string]
            
    for string in paths_surface_pressure:
                        
        if (current_year_string in string):
                
            year_limited_paths_surface_pressure += [string]
                
    for string in paths_precipitation:
                        
        if (current_year_string in string):
                
            year_limited_paths_precipitation += [string]
    

2010


In [12]:
 
    #####################
    ####  Load Data  ####
    #####################

    # Data is "lazy loaded", nothing is actually loaded until we "look" at data in some way #

    dataset_specific_humidity = xr.open_mfdataset(year_limited_paths_specific_humidity, combine="by_coords")
    dataset_temperature = xr.open_mfdataset(year_limited_paths_temperature, combine="by_coords")
    dataset_surface_pressure = xr.open_dataset(year_limited_paths_surface_pressure[0])
    dataset_precipitation = xr.open_mfdataset(year_limited_paths_precipitation, combine="by_coords")
    dataset_land = xr.open_dataset(paths_land[0])


ValueError: found the following matches with the input file in xarray's IO backends: ['netcdf4', 'h5netcdf']. But their dependencies may not be installed, see:
https://docs.xarray.dev/en/stable/user-guide/io.html 
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html

In [8]:

    #####################
    ####  Load Data  ####
    #####################
              
    # Make data arrays, loading only the year of interest #
    full_lat = dataset_surface_pressure['lat']
    full_lon = dataset_surface_pressure['lon']
    land_sea_mask = dataset_land['land_sea_mask']

    PS = dataset_surface_pressure['pres'].sel(time = slice(str(year)+'-01-01', str(year)+'-12-31'), lat = slice(15, -15)) # [Pa]
    Q = dataset_specific_humidity['shum'].sel(time = slice(str(year)+'-01-01', str(year)+'-12-31'),lat = slice(15, -15), level = slice(70, 1000)) # [Kg/Kg]
    T = dataset_temperature['air'].sel(time = slice(str(year)+'-01-01', str(year)+'-12-31'),lat = slice(15, -15), level = slice(70, 1000)) # [K]

    # Actually load data #
    land_sea_mask.load()
    PS.load()
    Q.load()
    T.load()

    # Clean up environment #
    
    gc.collect();
 


In [ ]:
    ################################
    ####  Average Data to Daily ####
    ################################
    
    ###   Test for Averaging Method   ###
            
    #PS.sel(time=slice('1998-01-01','1998-01-01'),lat=5,lon=75).plot()
    #print(PS.resample(time='1D').mean('time').sel(time=slice('1998-01-01','1998-01-01'),lat=5,lon=75))
    #print(PS.sel(time=slice('1998-01-01','1998-01-01'),lat=5,lon=75).mean('time'))
    #print(PS.resample(time='1D').mean('time').sel(time=slice('1998-01-01','1998-01-01'),lat=5,lon=75).values == PS.sel(time=slice('1998-01-01','1998-01-01'),lat=5,lon=75).mean('time').values)
    
    ###   Perform Averaging   ###
            
    PS = PS.resample(time='1D').mean('time')
    Q = Q.resample(time='1D').mean('time')
    T = T.resample(time='1D').mean('time')

    ### Update time to reflect center of daily average   ###
    
    PS = PS.assign_coords({'time':PS['time']+pd.to_timedelta(10.5, unit='H')})
    Q = Q.assign_coords({'time':Q['time']+pd.to_timedelta(10.5, unit='H')})
    T = T.assign_coords({'time':T['time']+pd.to_timedelta(10.5, unit='H')})


In [ ]:

    ###############################################
    ####  Modify "landfrac" Variable as Needed ####
    ###############################################
    
    print("Modifying landfrac as needed")

    landfrac = land_sea_mask.rename({'Latitude':'lat','Longitude':'lon'})
    landfrac = landfrac.rename('landfrac')
    print(landfrac)
    
    # The landfrac variable does not have lat/lon coordinates. Assign those of variables and check to make sure they make sense #
    
    #print(landfrac.coords['lat'])
    landfrac.coords['lat'] = full_lat.coords['lat']
    landfrac.coords['lon'] = full_lon.coords['lon']

    landfrac = landfrac.transpose()
    
    # Clean up environment #
    
    gc.collect();
    
    #####################################
    ####  Modify variables as needed ####
    #####################################
    
    PS = PS.rename('PS')
    PS = PS.transpose('time','lat','lon')
    PS = PS.sortby('lat', ascending=True) # Re-order lat to match code for other datasets
    #print(PS)
    
    Q = Q.rename({'level':'lev'})
    Q = Q.rename('Q')
    Q = Q.transpose('time','lev','lat','lon')
    Q = Q.sortby('lat', ascending=True) # Re-order lat to match code for other datasets
    #print(Q)
    
    T = T.rename({'level':'lev'})
    T = T.rename('T')
    T = T.transpose('time','lev','lat','lon')
    T = T.sortby('lat', ascending=True) # Re-order lat to match code for other datasets
    #print(T)
    
    landfrac = landfrac.sortby('lat', ascending=True) # Re-order lat to match code for other datasets
    
    # Clean up environment #
    
    gc.collect();

    #########################################
    ####  Calculate True Model Pressure  ####
    #########################################

    print("Calculating true model pressure")
    
    # Set upper most interface equal to uppermost level midpoint, and lowest interface equal to surface pressure.
    # This will still permit the desired vertical integral, just choose appropriate upper and lower integration limits
    
    # Model level midpoint

    true_pressure_midpoint = Q['lev'] * 100. # To convert to Pa
    true_pressure_midpoint = true_pressure_midpoint.rename('true_pressure_midpoint_Pa')
    true_pressure_midpoint = true_pressure_midpoint.expand_dims({'lat':Q['lat'], 'lon':Q['lon'], 'time':Q['time']})
    true_pressure_midpoint = true_pressure_midpoint.transpose('time','lev','lat','lon')
    
    # Model level interfaces
    
    true_pressure_interface = np.empty((len(Q.time),len(Q.lat),len(Q.lon),len(Q.lev)+1))

    for interface_level_counter in range(len(Q.lev) + 1):
        if interface_level_counter == 0:
            true_pressure_interface[:,:,:,interface_level_counter] = Q['lev'].isel(lev=0).values # Set upper most interface equal to uppermost level midpoint
        elif interface_level_counter == (len(Q.lev)):
            true_pressure_interface[:,:,:,interface_level_counter] = PS # Set lowest interface equal to surface pressure
        else:
            true_pressure_interface[:,:,:,interface_level_counter] = (Q['lev'].isel(lev=interface_level_counter-1).values + Q['lev'].isel(lev=interface_level_counter).values) / 2.,  # Set middle interfaces equal to half way points between level midpoints
            
    coords = {'time':Q['time'], 'lat':Q['lat'], 'lon':Q['lon'], 'ilev':np.arange(1,len(Q.lev) + 2)}
    dims = ['time', 'lat', 'lon', 'ilev']
    true_pressure_interface = xr.DataArray(true_pressure_interface,dims=dims,coords=coords) * 100. # To convert to Pa
    true_pressure_interface.attrs['units'] = 'Pa'      
    
    true_pressure_interface = true_pressure_interface.transpose('time','ilev','lat','lon')

    # Clean up environment #
    
    gc.collect();

    ###############################################
    ####  Limit to Oceanic (<10% Land) Points  ####
    ###############################################
        
    print('Applying Land/Ocean Mask')
        
    # Create ocean mask #

    is_valid_ocean_mask = (landfrac.sel(lat = slice(-15, 15)) < 0.1)

    #is_valid_ocean_mask.plot()

    # Apply ocean mask to appropriate variables, setting invalid locations to nan #

    PS_ocean = PS.where(is_valid_ocean_mask, other = np.nan)
    
    #####################################################
    ####  Instantiate Virtual Temperature Variables  ####
    #####################################################
    
    nan_data = np.zeros(np.shape(Q))
    nan_data[:] = np.nan
    coords = Q.coords
    dims = Q.dims
    
    temp_v_env_all_times = xr.DataArray(nan_data,coords=coords,dims=dims)
    temp_v_plume_DIB_all_times = xr.full_like(temp_v_env_all_times,np.nan) # can't simply repeat use of nan_data, otherwise pointer indicates same variable
    temp_v_plume_DIBDBL_all_times = xr.full_like(temp_v_env_all_times,np.nan) # can't simply repeat use of nan_data, otherwise pointer indicates same variable
    temp_v_plume_NOMIX_all_times = xr.full_like(temp_v_env_all_times,np.nan)
 
    # Name variables #

    temp_v_env_all_times.name = 'temp_v_env'
    temp_v_plume_DIB_all_times.name = 'temp_v_plume_DIB'
    temp_v_plume_DIBDBL_all_times.name = 'temp_v_plume_DIBDBL'
    temp_v_plume_NOMIX_all_times.name = 'temp_v_plume_NOMIX'
    
    # Add desired attributes #
    
    temp_v_env_all_times.attrs['Units'] = '[K]'
    temp_v_plume_DIB_all_times.attrs['Units'] = '[K]'
    temp_v_plume_DIBDBL_all_times.attrs['Units'] = '[K]'
    temp_v_plume_NOMIX_all_times.attrs['Units'] = '[K]'
    
    ######################################
    ####  Instantiate CAPE Variables  ####
    ######################################
    
    nan_data = np.zeros(np.shape(PS))
    nan_data[:] = np.nan
    coords = PS.coords
    dims = PS.dims
 
    CAPE_DIB_1000_to_600 = xr.DataArray(nan_data,coords=coords,dims=dims)
    CAPE_DIBDBL_1000_to_600 = xr.full_like(CAPE_DIB_1000_to_600,np.nan) # can't simply repeat use of nan_data, otherwise pointer indicates same variable
    CAPE_NOMIX_1000_to_600 = xr.full_like(CAPE_DIB_1000_to_600,np.nan) # can't simply repeat use of nan_data, otherwise pointer indicates same variable
    
    CAPE_DIB_1000_to_850 = xr.full_like(CAPE_DIB_1000_to_600,np.nan)
    CAPE_DIBDBL_1000_to_850 = xr.full_like(CAPE_DIB_1000_to_600,np.nan)
    CAPE_NOMIX_1000_to_850 = xr.full_like(CAPE_DIB_1000_to_600,np.nan)
    
    CAPE_DIB_850_to_600 = xr.full_like(CAPE_DIB_1000_to_600,np.nan)
    CAPE_DIBDBL_850_to_600 = xr.full_like(CAPE_DIB_1000_to_600,np.nan)
    CAPE_NOMIX_850_to_600 = xr.full_like(CAPE_DIB_1000_to_600,np.nan)
    
    # Name variables #

    CAPE_DIB_1000_to_600.name = 'CAPE_DIB_1000_to_600'
    CAPE_DIBDBL_1000_to_600.name = 'CAPE_DIBDBL_1000_to_600'
    CAPE_NOMIX_1000_to_600.name = 'CAPE_NOMIX_1000_to_600'
    
    CAPE_DIB_1000_to_850.name = 'CAPE_DIB_1000_to_850'
    CAPE_DIBDBL_1000_to_850.name = 'CAPE_DIBDBL_1000_to_850'
    CAPE_NOMIX_1000_to_850.name = 'CAPE_NOMIX_1000_to_850'
    
    CAPE_DIB_850_to_600.name = 'CAPE_DIB_850_to_600'
    CAPE_DIBDBL_850_to_600.name = 'CAPE_DIBDBL_850_to_600'
    CAPE_NOMIX_850_to_600.name = 'CAPE_NOMIX_850_to_600'
    
    # Add desired attributes #

    CAPE_DIB_1000_to_600.attrs['Units'] = '[J Kg^-1]'
    CAPE_DIBDBL_1000_to_600.attrs['Units'] = '[J Kg^-1]'
    CAPE_NOMIX_1000_to_600.attrs['Units'] = '[J Kg^-1]'
    
    CAPE_DIB_1000_to_850.attrs['Units'] = '[J Kg^-1]'
    CAPE_DIBDBL_1000_to_850.attrs['Units'] = '[J Kg^-1]'
    CAPE_NOMIX_1000_to_850.attrs['Units'] = '[J Kg^-1]'
    
    CAPE_DIB_850_to_600.attrs['Units'] = '[J Kg^-1]'
    CAPE_DIBDBL_850_to_600.attrs['Units'] = '[J Kg^-1]'
    CAPE_NOMIX_850_to_600.attrs['Units'] = '[J Kg^-1]'

    ####################################
    ####  Calculate Buoyancy Terms  ####
    ####################################
    
    launch_level_hPa = 1000 # [hPa] per Fiaz's code. Must remain close to 1000 to maintain mass flux profile
    
    for latitude in CAPE_DIB_1000_to_600.lat:
        
        print(latitude)
        
        for longitude in CAPE_DIB_1000_to_600.lon:
            
            if is_valid_ocean_mask.sel(lat=latitude,lon=longitude):
                
                ########################################
                ####  Virtual Temperature Variables ####
                ########################################
                
                temp_v_env, temp_v_plume_DIB, temp_v_plume_NOMIX, temp_v_plume_DIBDBL = numerical_plume_model(T.sel(lat=latitude,lon=longitude), Q.sel(lat=latitude,lon=longitude), 1000)
                
                temp_v_env_all_times.loc[dict(lat=latitude,lon=longitude)] = temp_v_env
                temp_v_plume_DIB_all_times.loc[dict(lat=latitude,lon=longitude)] = temp_v_plume_DIB
                temp_v_plume_DIBDBL_all_times.loc[dict(lat=latitude,lon=longitude)] = temp_v_plume_DIBDBL
                temp_v_plume_NOMIX_all_times.loc[dict(lat=latitude,lon=longitude)] = temp_v_plume_NOMIX

                ##########################
                ####  CAPE Variables  ####
                ##########################
    
                # print('Calculating CAPE Variables')
                
                CAPE_DIB_1000_to_600.loc[dict(lat=latitude,lon=longitude)] = calculate_CAPE(temp_v_env, temp_v_plume_DIB, true_pressure_midpoint.sel(lat=latitude,lon=longitude), true_pressure_interface.sel(lat=latitude,lon=longitude), 100000, 60000)
                
                CAPE_DIBDBL_1000_to_600.loc[dict(lat=latitude,lon=longitude)] = calculate_CAPE(temp_v_env, temp_v_plume_DIBDBL, true_pressure_midpoint.sel(lat=latitude,lon=longitude), true_pressure_interface.sel(lat=latitude,lon=longitude), 100000, 60000)

                CAPE_NOMIX_1000_to_600.loc[dict(lat=latitude,lon=longitude)] = calculate_CAPE(temp_v_env, temp_v_plume_NOMIX, true_pressure_midpoint.sel(lat=latitude,lon=longitude), true_pressure_interface.sel(lat=latitude,lon=longitude), 100000, 60000)
                
                
                CAPE_DIB_1000_to_850.loc[dict(lat=latitude,lon=longitude)] = calculate_CAPE(temp_v_env, temp_v_plume_DIB, true_pressure_midpoint.sel(lat=latitude,lon=longitude), true_pressure_interface.sel(lat=latitude,lon=longitude), 100000, 85000)
                
                CAPE_DIBDBL_1000_to_850.loc[dict(lat=latitude,lon=longitude)] = calculate_CAPE(temp_v_env, temp_v_plume_DIBDBL, true_pressure_midpoint.sel(lat=latitude,lon=longitude), true_pressure_interface.sel(lat=latitude,lon=longitude), 100000, 85000)

                CAPE_NOMIX_1000_to_850.loc[dict(lat=latitude,lon=longitude)] = calculate_CAPE(temp_v_env, temp_v_plume_NOMIX, true_pressure_midpoint.sel(lat=latitude,lon=longitude), true_pressure_interface.sel(lat=latitude,lon=longitude), 100000, 85000)
                
                
                CAPE_DIB_850_to_600.loc[dict(lat=latitude,lon=longitude)] = calculate_CAPE(temp_v_env, temp_v_plume_DIB, true_pressure_midpoint.sel(lat=latitude,lon=longitude), true_pressure_interface.sel(lat=latitude,lon=longitude), 85000, 60000)
                
                CAPE_DIBDBL_850_to_600.loc[dict(lat=latitude,lon=longitude)] = calculate_CAPE(temp_v_env, temp_v_plume_DIBDBL, true_pressure_midpoint.sel(lat=latitude,lon=longitude), true_pressure_interface.sel(lat=latitude,lon=longitude), 85000, 60000)

                CAPE_NOMIX_850_to_600.loc[dict(lat=latitude,lon=longitude)] = calculate_CAPE(temp_v_env, temp_v_plume_NOMIX, true_pressure_midpoint.sel(lat=latitude,lon=longitude), true_pressure_interface.sel(lat=latitude,lon=longitude), 85000, 60000)    
        
                # Clean up environment #
    
                gc.collect();
      
    #################################
    ####  Output Data as NetCDF  ####
    #################################

    # Name variables #

    landfrac.name = 'landfrac'
    
    # Add desired attributes #

    landfrac.attrs['Units'] = 'Fraction of land, 0 = all water, 1 = all land'
                                                       
    # Merge all neccessary dataarrays to a single dataset #

    output_dataset = xr.merge([landfrac, CAPE_DIB_1000_to_600, CAPE_DIBDBL_1000_to_600, CAPE_NOMIX_1000_to_600,\
                              CAPE_DIB_1000_to_850, CAPE_DIBDBL_1000_to_850, CAPE_NOMIX_1000_to_850,\
                              CAPE_DIB_850_to_600, CAPE_DIBDBL_850_to_600, CAPE_NOMIX_850_to_600])

    # Output dataset to NetCDF #

    output_dataset.to_netcdf(fname_datasets[0] + '_' + str(year) + '.nc')
    
    temp_v_env_all_times.to_netcdf(fname_datasets_temp_v_env[0] + '_' + str(year) + '.nc')
    temp_v_plume_DIB_all_times.to_netcdf(fname_datasets_temp_v_plume_DIB[0] + '_' + str(year) + '.nc')
    temp_v_plume_DIBDBL_all_times.to_netcdf(fname_datasets_temp_v_plume_DIB[0] + '_' + str(year) + '.nc')
    temp_v_plume_NOMIX_all_times.to_netcdf(fname_datasets_temp_v_plume_NOMIX[0] + '_' + str(year) + '.nc')
    
    ########################
    ###  Calculate CSF  ####
    ########################
    
    ####  Calculate Saturation Specific Humidity  ####

    print("Calculating saturation specific humidity")

    saturation_specific_humidity = xr.apply_ufunc(calculate_saturation_specific_humidity, true_pressure_midpoint, T,
                                            output_dtypes=[Q.dtype])

    #saturation_specific_humidity_metpy = xr.apply_ufunc(calculate_saturation_specific_humidity_metpy, true_pressure_midpoint, T,
    #                                     output_dtypes=[Q.dtype])
    
    # Clean up environment #
    
    gc.collect();

    ####  Column Integrate Variables  ####
        
    upper_level_integration_limit_Pa = 10000 # [Pa]
        
    lower_level_integration_limit_Pa = 100000 # [Pa]

    print('Column Integrating')
        
    ci_q, _, _ = mass_weighted_vertical_integral_w_nan(Q, true_pressure_midpoint, true_pressure_interface, lower_level_integration_limit_Pa, upper_level_integration_limit_Pa)
    #print(ci_q)
    #print(ci_q.min())
    #print(ci_q.max())
    #print(ci_q.mean())
    #plt.figure()
    #ci_q.isel(time = 0).plot()
        
    ci_q_sat, _, _ = mass_weighted_vertical_integral_w_nan(saturation_specific_humidity, true_pressure_midpoint, true_pressure_interface, lower_level_integration_limit_Pa, upper_level_integration_limit_Pa)
    #print(ci_q_sat)
    #print(ci_q_sat.min())
    #print(ci_q_sat.max())
    #print(ci_q_sat.mean())
    #plt.figure()
    #ci_q_sat.isel(time = 0).plot()
        
    csf = ci_q / ci_q_sat
    print(csf)
    print(csf.min())
    print(csf.max())
    plt.figure()
    csf.isel(time = 0).plot()
    
    # Name variables #

    csf.name = 'csf'
    csf.attrs['Units'] = '[Kg Kg^-1]'

    # Clean up environment #
        
    gc.collect();
    
    #################################
    ####  Output Data as NetCDF  ####
    #################################

    # Output dataset to NetCDF #

    csf.to_netcdf(fname_datasets_CSF[0] + '_' + str(year) + '.nc')